In [1]:
import sys
sys.path.append('..')
from swdf.benchmark import *
from tsai.basics import *
import sklearn
from swdf.utils import *
import wandb
wandb_api = wandb.Api()

/usr/local/pip-global/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# Config
config = yaml2dict('config/solfsmy.yaml')
config = config.eval

config_base = yaml2dict('config/base.yaml')

config, config_base

({'dst_data_path': '../data/DST_IAGA2002.txt',
  'learner_artifact': None,
  'round_preds': True,
  'solfsmy_data_path': '../data/SOLFSMY.TXT'},
 {'arch_name': 'PatchTST',
  'arch_config_file': 'config/patchtst.yaml',
  'bs': 1024,
  'horizon': 6,
  'init_weights': False,
  'lookback': 36,
  'lr_max': None,
  'n_epoch': 5,
  'partial_n': 0.1,
  'seed': 42,
  'test_start_datetime': '2012-10-01 12:00:00',
  'test_end_datetime': '2018-12-31 12:00:00',
  'wandb': {'enabled': False,
   'log_learner': True,
   'mode': 'offline',
   'group': None,
   'project': 'swdf'}})

In [3]:
learn = load_learner_all("tmp", model_fname = 'model', 
                         verbose=True, 
                         device=default_device())

Learner loaded:
path          = 'tmp'
dls_fname     = '['dls_0.pth', 'dls_1.pth', 'dls_2.pth', 'dls_3.pth', 'dls_4.pth', 'dls_5.pth', 'dls_6.pth']'
model_fname   = 'model.pth'
learner_fname = 'learner.pkl'


In [4]:
y_test_preds, y_test = learn.get_preds(ds_idx = 2, with_targs=True)
y_test_preds = to_np(y_test_preds)
y_test = to_np(y_test)
print(f"y_test_preds.shape: {y_test_preds.shape}")    

y_test_preds.shape: (408, 4, 3)


In [ ]:
horizon = y_test.shape[-1]
data_columns_fcst = ['F10', 'S10', 'M10', 'Y10']

In [ ]:
y_test_split, y_test_preds_split, y_persistence_split, y_persistence_preds_split = {}, {}, {}, {}
for i, var in enumerate(data_columns_fcst):
    y_test_split[var] = y_test[:,i,:]
    y_test_preds_split[var] = y_test_preds[:,i,:]
    y_persistence_split[var] = y_persistence[:,i,:]
    y_persistence_preds_split[var] = y_persistence_preds[:,i,:]


# Check the shape of each variable
for var in data_columns_fcst:
    print(f"{var}: {[y_test_split[var].shape]}, {[y_test_preds_split[var].shape]}, {[y_persistence_split[var].shape]}, {[y_persistence_preds_split[var].shape]}")